In [50]:
import pandas as pd
import yfinance as yf
import ta
import seaborn as sns
import conf

# Data Pulling

In [51]:
df = pd.DataFrame()

In [52]:
for ticker in conf.TICKERS:
    temp_df = yf.download(ticker,
                    start='2004-08-19',
                    end='2023-07-01')
    temp_df['ticker'] = ticker
    df = pd.concat([df,temp_df])
df = df.reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [53]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2004-08-19,2.013000,2.024000,1.817500,1.931500,1.931500,253922000,AMZN
1,2004-08-20,1.923000,1.995500,1.905500,1.975500,1.975500,135816000,AMZN
2,2004-08-23,1.994500,2.000000,1.955500,1.972500,1.972500,110652000,AMZN
3,2004-08-24,1.986000,1.996500,1.916000,1.952500,1.952500,152808000,AMZN
4,2004-08-25,1.953000,2.024500,1.908000,2.015000,2.015000,145096000,AMZN
...,...,...,...,...,...,...,...,...
52234,2023-06-26,74.089996,74.790001,73.809998,74.290001,74.290001,5670200,NEE
52235,2023-06-27,74.559998,74.800003,73.209999,73.910004,73.910004,5979100,NEE
52236,2023-06-28,73.949997,74.209999,73.220001,73.440002,73.440002,8735100,NEE
52237,2023-06-29,72.680000,73.599998,72.510002,72.809998,72.809998,7193200,NEE


In [54]:
df.columns = df.columns.str.lower()
df = df[['date', 'ticker', 'open', 'high', 'low', 'close', 'adj close', 'volume']]
df['close'] = df['adj close']
df = df.drop(columns='adj close')
df['day'] = df['date'].dt.day_of_week
df = df.sort_values(['date', 'ticker']).reset_index(drop=True)

In [55]:
df.to_csv('raw_data.csv')

# Add Technical Analysis

In [56]:
def get_ta(df):
    df['sma'] = ta.trend.sma_indicator(df['close'], window=9)
    df['ema'] = ta.trend.ema_indicator(df['close'], window=9)
    df['rsi'] = ta.momentum.rsi(df['close'])
    df['macd'] = ta.trend.macd_diff(df['close'])
    df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    df['cci'] = ta.trend.cci(df['high'], df['low'], df['close'])
    df['adx'] = ta.trend.adx(df['high'], df['low'], df['close'])
    df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'])
    df.dropna(inplace=True)

    return df

In [68]:
ta_df = pd.DataFrame()

In [69]:
for ticker in df['ticker'].unique():
    temp_df = get_ta(df[df['ticker'] == ticker])
    ta_df = pd.concat([ta_df,temp_df])

C:\Users\nolan\AppData\Local\Temp\ipykernel_18804\4224848282.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sma'] = ta.trend.sma_indicator(df['close'], window=9)
C:\Users\nolan\AppData\Local\Temp\ipykernel_18804\4224848282.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ema'] = ta.trend.ema_indicator(df['close'], window=9)
C:\Users\nolan\AppData\Local\Temp\ipykernel_18804\4224848282.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [70]:
ta_df

,date,ticker,open,high,low,close,volume,day,sma,ema,rsi,macd,obv,cci,adx,atr
363,2004-10-06,AAPL,0.705357,0.727857,0.704821,0.616815,446303200,2,0.586461,0.589146,81.920822,0.001728,6535653600,167.336208,70.660203,0.110120
374,2004-10-07,AAPL,0.723929,0.730893,0.704643,0.601334,426148800,3,0.590390,0.591583,69.802763,0.001194,6109504800,134.752724,70.987939,0.110403
385,2004-10-08,AAPL,0.706429,0.710179,0.693571,0.592835,359228800,4,0.592970,0.591834,64.188760,0.000065,5750276000,81.428997,70.056710,0.110292
396,2004-10-11,AAPL,0.692857,0.697500,0.682143,0.585702,323870400,0,0.593897,0.590607,59.838899,-0.001292,5426405600,41.163785,68.076560,0.109890
407,2004-10-12,AAPL,0.687500,0.688929,0.672321,0.581148,460191200,1,0.593240,0.588715,57.175385,-0.002553,4966214400,5.866547,65.385496,0.109414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52194,2023-06-26,XOM,102.199997,104.910004,102.199997,104.290001,14283400,0,104.355557,104.232726,46.014304,-0.143732,682949600,-56.839252,19.175453,2.377816
52205,2023-06-27,XOM,104.050003,104.610001,103.269997,104.550003,13227500,1,104.145557,104.296181,46.895635,-0.054730,696177100,-38.128023,18.975234,2.303687
52216,2023-06-28,XOM,104.309998,105.820000,103.199997,105.400002,13084100,2,104.172223,104.516945,49.781966,0.065820,709261200,-13.254607,18.127946,2.326281
52227,2023-06-29,XOM,105.660004,106.730003,105.129997,106.699997,11364400,3,104.262223,104.953556,53.908149,0.228392,720625600,42.549070,16.877988,2.274404


In [71]:
ta_df = ta_df.sort_values(['date', 'ticker']).reset_index(drop=True)
ta_df.index = ta_df.date.factorize()[0]
ta_df

,date,ticker,open,high,low,close,volume,day,sma,ema,rsi,macd,obv,cci,adx,atr
0,2004-10-06,AAPL,0.705357,0.727857,0.704821,0.616815,446303200,2,0.586461,0.589146,81.920822,0.001728,6535653600,167.336208,70.660203,0.110120
0,2004-10-06,AMZN,2.036500,2.070500,2.020000,2.068000,102562000,2,2.032222,2.048355,55.424910,-0.006109,542660000,-3.493600,18.561893,0.065289
0,2004-10-06,BRK-B,57.160000,57.299999,56.720001,56.820000,1175000,2,57.215555,57.153657,39.513774,-0.098448,-2725000,-162.113133,11.037878,0.458926
0,2004-10-06,GOOG,3.425909,3.448325,3.387304,3.414203,268900140,2,3.234321,3.256696,79.392864,0.035793,4412794472,132.316169,46.875097,0.115029
0,2004-10-06,LIN,42.750000,43.180000,42.750000,29.916002,851900,2,29.392937,29.526530,68.540184,0.065223,4653000,133.042297,35.237224,13.028961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,2023-06-30,PLD,122.550003,122.879997,119.910004,122.629997,6598700,4,120.188889,120.909493,53.214996,0.153611,661042700,1.740759,11.749923,2.808280
4715,2023-06-30,UNH,478.000000,482.100006,475.119995,480.640015,3970100,4,477.260003,477.604876,50.760942,0.616983,584102300,-6.637103,15.197652,9.895974
4715,2023-06-30,UPS,176.589996,179.649994,175.559998,179.250000,4891700,4,174.438890,175.229139,60.713882,0.648649,-1500000,118.022173,16.178081,3.098152
4715,2023-06-30,WMT,154.970001,157.529999,154.779999,157.179993,6357000,4,155.128891,155.223229,64.128756,0.025758,963892500,83.865890,20.646912,1.940878


In [72]:
cov_list = []
lookback = 252*5

for i in range(lookback, len(ta_df.index.unique())):
    data_lookback = ta_df.loc[i-lookback:i,:]
    price_lookback = data_lookback.pivot_table(index='date', columns='ticker', values='close')
    return_lookback = price_lookback.pct_change().dropna()
    cov = return_lookback.cov().values
    cov_list.append(cov)

In [73]:
df_cov = pd.DataFrame({'date':ta_df.date.unique()[lookback:], 'cov_list':cov_list})
ta_df = ta_df.merge(df_cov, on='date')
ta_df = ta_df.sort_values(['date', 'ticker']).reset_index(drop=True)


,date,ticker,open,high,low,close,volume,day,sma,ema,rsi,macd,obv,cci,adx,atr,cov_list
38011,2023-06-30,PLD,122.550003,122.879997,119.910004,122.629997,6598700,4,120.188889,120.909493,53.214996,0.153611,661042700,1.740759,11.749923,2.808280,"[[0.0004399161902258871, 0.0003132764645333177..."
38012,2023-06-30,UNH,478.000000,482.100006,475.119995,480.640015,3970100,4,477.260003,477.604876,50.760942,0.616983,584102300,-6.637103,15.197652,9.895974,"[[0.0004399161902258871, 0.0003132764645333177..."
38013,2023-06-30,UPS,176.589996,179.649994,175.559998,179.250000,4891700,4,174.438890,175.229139,60.713882,0.648649,-1500000,118.022173,16.178081,3.098152,"[[0.0004399161902258871, 0.0003132764645333177..."
38014,2023-06-30,WMT,154.970001,157.529999,154.779999,157.179993,6357000,4,155.128891,155.223229,64.128756,0.025758,963892500,83.865890,20.646912,1.940878,"[[0.0004399161902258871, 0.0003132764645333177..."
38015,2023-06-30,XOM,107.320000,107.879997,106.809998,107.250000,13542700,4,104.497779,105.412844,55.571405,0.360397,734168300,90.373220,16.174652,2.196232,"[[0.0004399161902258871, 0.0003132764645333177..."


In [74]:
ta_df.head(22)

,date,ticker,open,high,low,close,volume,day,sma,ema,rsi,macd,obv,cci,adx,atr,cov_list
0,2009-10-07,AAPL,6.777143,6.805357,6.751071,5.775057,465668000,2,5.636907,5.651985,69.783051,0.000103,76590477600,120.587879,25.406273,1.043163,"[[0.0007465746450822919, 0.0003705057707397622..."
1,2009-10-07,AMZN,4.575000,4.724000,4.557500,4.698500,148902000,2,4.568056,4.562287,62.868703,-0.002230,7308406000,95.314809,21.863590,0.125389,"[[0.0007465746450822919, 0.0003705057707397622..."
2,2009-10-07,BRK-B,65.199997,65.940002,65.199997,65.800003,1245000,2,65.805556,65.858521,50.348832,-0.140368,-4180000,-42.923092,13.177895,1.217660,"[[0.0007465746450822919, 0.0003705057707397622..."
3,2009-10-07,GOOG,12.428415,12.926299,12.398776,12.890184,195698788,2,12.348077,12.384596,73.462918,0.003989,22293473363,166.673276,34.746270,0.242156,"[[0.0007465746450822919, 0.0003705057707397622..."
4,2009-10-07,LIN,80.629997,81.500000,80.449997,61.233204,1603200,2,60.827836,60.801278,58.217536,-0.094041,57375000,66.892637,14.658829,20.083793,"[[0.0007465746450822919, 0.0003705057707397622..."
5,2009-10-07,NEE,13.437500,13.465000,13.195000,8.850360,15136400,2,9.053391,9.010343,36.661175,-0.013793,593872000,-135.064744,18.540362,4.633883,"[[0.0007465746450822919, 0.0003705057707397622..."
6,2009-10-07,PLD,21.330000,21.860001,21.180000,14.040749,1533300,2,14.349321,14.325510,45.142651,-0.196425,129839800,-105.247105,19.854168,8.517906,"[[0.0007465746450822919, 0.0003705057707397622..."
7,2009-10-07,UNH,24.420000,25.340000,24.400000,20.485798,12081700,2,20.390386,20.562127,39.188579,-0.213681,-558561100,-74.459319,31.051380,5.302521,"[[0.0007465746450822919, 0.0003705057707397622..."
8,2009-10-07,UPS,55.500000,55.799999,55.150002,36.865707,2349600,2,37.159495,37.166267,46.645323,-0.302220,-245837200,-99.992432,22.329695,19.494132,"[[0.0007465746450822919, 0.0003705057707397622..."
9,2009-10-07,WMT,49.529999,49.700001,49.320000,36.332489,11620200,2,36.168530,36.310108,43.060697,-0.057928,-58863600,-52.807551,19.977935,13.593119,"[[0.0007465746450822919, 0.0003705057707397622..."


In [75]:
ta_df.to_csv("preprocessed.csv")